In [ ]:
# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [27]:
# import libraries
import pandas as pd 
import numpy as np 
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import Pipeline
from sqlalchemy import create_engine
import nltk


In [28]:
nltk.download(["wordnet",'punkt', 'stopwords'])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('messages_table' , engine)
df.shape

(26216, 40)

In [30]:
#df =df.fillna(method ='ffill')
#df.shape

In [31]:
X=df[['message', 'id' ,'original' , 'genre']]
X.head()

,message,id,original,genre
0,Weather update - a cold front from Cuba that c...,2,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,Is the Hurricane over or is it not over,7,Cyclone nan fini osinon li pa fini,direct
2,Looking for someone but no name,8,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,UN reports Leogane 80-90 destroyed. Only Hospi...,9,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,"says: west side of Haiti, rest of the country ...",12,facade ouest d Haiti et le reste du pays aujou...,direct


In [32]:
y = df.drop(['message','genre', 'id', 'original'], axis =1)
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
y['related']=y['related'].map(lambda x: 1 if x == 2 else x)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data


https://datascience.stackexchange.com/questions/53448/nltks-stopwords-returns-typeerror-argument-of-type-lazycorpusloader-is-not

In [34]:
#def tokenize(text):
    #tokens = nltk.word_tokenize(text)
    #lemmatizer = nltk.WordNetLemmatizer()
    #return [lemmatizer.lemmatize(w).lower().strip() for w in tokens]

In [35]:
import re 
def tokenize(text):
    text = re.sub(r'[^a-zA-Z0-9]',' ',text.lower())
    words = nltk.word_tokenize(text)
    tokens = [w for w in words if w not in stopwords.words("english")]
    lemmatizer = nltk.WordNetLemmatizer()
    return [lemmatizer.lemmatize(w).lower().strip() for w in tokens]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [36]:
from sklearn.multioutput import MultiOutputClassifier

pipeline = Pipeline([('cvect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', RandomForestClassifier())
                     ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit (X_train['message'], y_train )

Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [38]:
from sklearn.metrics import classification_report


y_pred =  pipeline.predict(X_test['message'])
for i, col in enumerate(y_test):
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))

related
             precision    recall  f1-score   support

          0       0.61      0.50      0.55      1599
          1       0.85      0.90      0.87      4955

avg / total       0.79      0.80      0.79      6554

request
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5477
          1       0.81      0.42      0.55      1077

avg / total       0.88      0.89      0.87      6554

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.71      0.92      0.80      3878
          1       0.79      0.45      0.57      2676

avg / total       0.74      0.73      0.71      6554

medical_help
             precision    recall  f1-score   support

          0       0.93      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [39]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'cvect', 'tfidf', 'clf', 'cvect__analyzer', 'cvect__binary', 'cvect__decode_error', 'cvect__dtype', 'cvect__encoding', 'cvect__input', 'cvect__lowercase', 'cvect__max_df', 'cvect__max_features', 'cvect__min_df', 'cvect__ngram_range', 'cvect__preprocessor', 'cvect__stop_words', 'cvect__strip_accents', 'cvect__token_pattern', 'cvect__tokenizer', 'cvect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__bootstrap', 'clf__class_weight', 'clf__criterion', 'clf__max_depth', 'clf__max_features', 'clf__max_leaf_nodes', 'clf__min_impurity_decrease', 'clf__min_impurity_split', 'clf__min_samples_leaf', 'clf__min_samples_split', 'clf__min_weight_fraction_leaf', 'clf__n_estimators', 'clf__n_jobs', 'clf__oob_score', 'clf__random_state', 'clf__verbose', 'clf__warm_start'])

In [40]:
sorted(pipeline.get_params().keys())

['clf',
 'clf__bootstrap',
 'clf__class_weight',
 'clf__criterion',
 'clf__max_depth',
 'clf__max_features',
 'clf__max_leaf_nodes',
 'clf__min_impurity_decrease',
 'clf__min_impurity_split',
 'clf__min_samples_leaf',
 'clf__min_samples_split',
 'clf__min_weight_fraction_leaf',
 'clf__n_estimators',
 'clf__n_jobs',
 'clf__oob_score',
 'clf__random_state',
 'clf__verbose',
 'clf__warm_start',
 'cvect',
 'cvect__analyzer',
 'cvect__binary',
 'cvect__decode_error',
 'cvect__dtype',
 'cvect__encoding',
 'cvect__input',
 'cvect__lowercase',
 'cvect__max_df',
 'cvect__max_features',
 'cvect__min_df',
 'cvect__ngram_range',
 'cvect__preprocessor',
 'cvect__stop_words',
 'cvect__strip_accents',
 'cvect__token_pattern',
 'cvect__tokenizer',
 'cvect__vocabulary',
 'memory',
 'steps',
 'tfidf',
 'tfidf__norm',
 'tfidf__smooth_idf',
 'tfidf__sublinear_tf',
 'tfidf__use_idf']

In [53]:
def build_model():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', AdaBoostClassifier())
    ])

    parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }

    cv = GridSearchCV(pipeline, param_grid=parameters)

    return cv

In [54]:
parameters = {'clf__max_depth': [10, 20, None],
              'clf__min_samples_leaf': [1, 2, 4],
              'clf__min_samples_split': [2, 5, 10],
              'clf__n_estimators': [10, 20, 40]}

cv = GridSearchCV(pipeline, param_grid=parameters)
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__max_depth': [10, 20, None], 'clf__min_samples_leaf': [1, 2, 4], 'clf__min_samples_split': [2, 5, 10], 'clf__n_estimators': [10, 20, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [59]:
parameters = {
    'cvect__ngram_range': ((1, 1), (1, 2))
}

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=3, scoring='f1_weighted', verbose=3)

In [60]:
cv.fit(X_train['message'], y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] cvect__ngram_range=(1, 1) .......................................


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[CV]  cvect__ngram_range=(1, 1), score=0.4801877124747496, total= 1.9min
[CV] cvect__ngram_range=(1, 1) .......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.1min remaining:    0.0s
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[CV]  cvect__ngram_range=(1, 1), score=0.4924674191544521, total= 1.9min
[CV] cvect__ngram_range=(1, 1) .......................................


...

[CV]  cvect__ngram_range=(1, 2), score=0.46847123896934845, total= 2.2min
[CV] cvect__ngram_range=(1, 2) .......................................


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[CV]  cvect__ngram_range=(1, 2), score=0.4801758992917336, total= 2.3min


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 19.5min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'cvect__ngram_range': ((1, 1), (1, 2))},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_weighted', verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [61]:
y_pred =  pipeline.predict(X_test['message'])

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [62]:
cv.best_params_


{'cvect__ngram_range': (1, 1)}

### 9. Export your model as a pickle file

In [63]:
m = pickle.dumps('clf')


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.